In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.applications import resnet50
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import RMSprop

In [2]:
tf.__version__

'2.0.0'

In [13]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def siam_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''    
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [14]:
def create_base_model(input_shape, num_classes):
    image_input = Input(shape=input_shape)
    model = resnet50.ResNet50(weights="imagenet", include_top=True,
                          input_tensor=image_input)
    # 2048-D vector output
    embd_output = model.get_layer('avg_pool').output
    # can add more dense layers inbetween if required
    classification_output = Dense(num_classes, activation='softmax', name='output_layer')(embd_output)
    custom_resnet_model = Model(inputs=image_input, outputs= [embd_output, classification_output])
    return custom_resnet_model

def create_siamese_model(input_shape, num_classes):
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    model = create_base_model(input_shape, num_classes)
    embd_a, class_a = model(input_a)
    embd_b, class_b = model(input_b)
    # l2 norm for embeddings
    norm_embd_a = K.l2_normalize(embd_a, axis=1)
    norm_embd_b = K.l2_normalize(embd_b, axis=1)
    # distance between embeddings
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([norm_embd_a, norm_embd_b])
    custom_siamese_model = Model([input_a, input_b], [distance, class_a, class_b])
    return custom_siamese_model

In [15]:
input_shape = (224, 224, 3)
num_classes = 46

In [16]:
s_model = create_siamese_model(input_shape, num_classes)

In [17]:
s_model.output_names

['lambda_1', 'model_2', 'model_2_1']

In [18]:
output_names = s_model.output_names
losses = {
    output_names[0]: contrastive_loss,
    output_names[1]: "categorical_crossentropy",
    output_names[2]: "categorical_crossentropy",
}
lossWeights = {output_names[0]:1.0,output_names[1]: 1.0, output_names[2]: 1.0}

#top k accuracy would be better i guess
all_metrics = {
    output_names[0]: siam_accuracy,
    output_names[1]: "accuracy",
    output_names[2]: "accuracy",
}

In [19]:
rms = RMSprop()
s_model.compile(loss=losses, loss_weights=lossWeights, optimizer=rms, metrics=all_metrics)